In [1]:
# Installation step as requested (assuming necessary packages are not pre-installed)
%pip install numpy pandas json matplotlib bioverse==1.1.8

ERROR: Could not find a version that satisfies the requirement json (from versions: none)
ERROR: No matching distribution found for json

[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


## Magma Ocean Test Notebook

This notebook demonstrates how to simulate the detectability of magma ocean worlds, replacing visualization steps with data saving for a test environment.

## Imports & preparation

In [2]:
import numpy as np
import pandas as pd
import json # Added for saving analysis results

from bioverse.generator import Generator
from bioverse.survey import TransitSurvey
from bioverse.constants import CONST, ROOT_DIR

# Set a seed for reproducibility
np.random.seed(42)

# Import pyplot (only needed for setting font size, but kept for context)
from matplotlib import pyplot as plt
plt.rcParams['font.size'] = 20.

## Generator and Survey Configuration

We will load the standard Generator and a survey, and then define a custom step to calculate the magma ocean phase for planets.

In [3]:
# Define the custom function to calculate magma ocean phase
magmaocean_func = """
import numpy as np
from bioverse.constants import CONST
from bioverse.util import calc_inc

def magmaocean(d):
    # Calculate the effective temperature (Teq) as a proxy for magma ocean presence
    # Teq \propto (L_st / a^2)^{1/4}
    Teq = CONST['T_sun'] * (d['R_st'] / d['a'])**0.5 * d['T_eff'] / CONST['T_sun'] * (d['L_st'] / 1)**0.25
    
    # Simplified, approximate threshold for magma ocean phase (Teq > ~1500 K)
    T_mo_threshold = 1500 # K
    d['is_magmaocean'] = Teq > T_mo_threshold
    d['Teq'] = Teq

    # Assign a random CO2 feature strength for detected magma ocean worlds
    # This is a placeholder for a complex atmospheric model
    d['CO2_strength'] = np.zeros(len(d))
    is_mo = d['is_magmaocean']
    d['CO2_strength'][is_mo] = np.random.uniform(50, 500, size=is_mo.sum()) # ppm-level feature strength
    
    # Calculate the expected SNR for the CO2 feature based on transit depth
    # SNR is proportional to (R/Rst)^2 / sigma_depth. This is a further simplification.
    transit_depth = (d['R'] / d['R_st'])**2
    # Assume a constant noise level (sigma_depth) for simplicity in this test
    sigma_depth = 1e-4 
    d['SNR_CO2_feature'] = d['CO2_strength'] / (sigma_depth * 1e6) 
    
    return d
"""

# Save the function to a .py file so the Generator can load it
save_path = os.path.join(ROOT_DIR, 'example_magmaocean.py')
with open(save_path, 'w') as f:
    f.write(magmaocean_func)
print("Custom function saved to example_magmaocean.py")

# Load the Generator
generator = Generator('transit')

# Insert the custom step
generator.insert_step('magmaocean', filename='example_magmaocean.py')
print("Generator step 'magmaocean' inserted.")

<>:2: SyntaxWarning: invalid escape sequence '\p'
<>:2: SyntaxWarning: invalid escape sequence '\p'
/var/folders/ln/j3sg9p0d6_j7ky99sycs_13h0000gp/T/ipykernel_76726/4131876010.py:2: SyntaxWarning: invalid escape sequence '\p'
  magmaocean_func = """
/var/folders/ln/j3sg9p0d6_j7ky99sycs_13h0000gp/T/ipykernel_76726/4131876010.py:2: SyntaxWarning: invalid escape sequence '\p'
  magmaocean_func = """


NameError: name 'os' is not defined

In [3]:
# Load a Transit Survey (JWST-like, but with a specific exposure time for this test)
survey = TransitSurvey('default')
survey.set_arg('t_total', 365.25 * 3) # 3 years of total time
survey.set_arg('t_obs', 5) # 5 transits for each target
survey.set_arg('d_max', 100) # Max distance in parsecs

## Running the Simulation and Analysis

We run the generator to get a planetary sample, then use a simplified detection criterion for magma ocean worlds: detection occurs if the planet is a magma ocean world AND the CO2 feature SNR is above a threshold.

In [4]:
sample = generator.generate(d_max=survey.get_arg('d_max'))

is_mo = sample['is_magmaocean']
print(f"Total planets generated: {len(sample)}")
print(f"Total magma ocean planets: {is_mo.sum()}")

# --- 1. Filter and Save Magma Ocean Planet Data ---
magmaocean_planets = sample[is_mo]
try:
    df_magmaocean = magmaocean_planets.to_pandas()
except AttributeError:
    df_magmaocean = pd.DataFrame(magmaocean_planets)

# Keep only relevant columns for saving
cols_to_save = ['R_st', 'M_st', 'T_eff', 'P', 'R', 'Teq', 'CO2_strength', 'SNR_CO2_feature']
df_magmaocean[cols_to_save].to_csv('magmaocean_planets_data.csv', index=False)
print("Magma ocean planet properties saved to magmaocean_planets_data.csv")

# --- 2. Determine Detections and Save Summary ---
# Simplified detection based on CO2 SNR > a threshold
SNR_threshold = 5.0 # Required SNR for a 5-sigma detection
is_detected = is_mo & (sample['SNR_CO2_feature'] > SNR_threshold)
N_detected = is_detected.sum()

detection_summary = {
    'N_magmaocean_planets': int(is_mo.sum()),
    'SNR_threshold': float(SNR_threshold),
    'N_magmaocean_detected': int(N_detected),
    'detection_fraction': float(N_detected / is_mo.sum()) if is_mo.sum() > 0 else 0.0,
    'max_CO2_SNR': float(sample['SNR_CO2_feature'].max()) if len(sample)>0 else 0.0
}

output_filename_summary = 'magmaocean_detection_summary.json'
with open(output_filename_summary, 'w') as f:
    json.dump(detection_summary, f, indent=4)
print(f"Detection summary saved to {output_filename_summary}")

Total planets generated: 2831
Total magma ocean planets: 39
Magma ocean planet properties saved to magmaocean_planets_data.csv
Detection summary saved to magmaocean_detection_summary.json


The saved data and summary can be used to test the simulation's results and verify the performance of the custom `magmaocean` step and the simple detection criterion.

## Cleanup

The following lines of code will clean up the files created during this exercise:

In [5]:
import os
trash = [
    os.path.join(ROOT_DIR, 'example_magmaocean.py'),
    'magmaocean_planets_data.csv',
    'magmaocean_detection_summary.json'
]
for filename in trash:
    if os.path.exists(filename):
        os.remove(filename)
        print(f"Cleaned up: {filename}")

Cleaned up: example_magmaocean.py
Cleaned up: magmaocean_planets_data.csv
Cleaned up: magmaocean_detection_summary.json
